In [1]:
# First setup client
from pprint import pprint

from src.client import ExchangeClient
from src.request_utils import load_private_key_from_file

In [2]:
from src.params import *

prod_private_key = load_private_key_from_file(KALSHI_KEY)
exchange_client = ExchangeClient(exchange_api_base=PROD_API_BASE, key_id = API_ID, private_key = prod_private_key)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

{'exchange_active': True, 'trading_active': True}


In [8]:
# First get the trading prices for the deep-blue bet
response = exchange_client.get_markets(series_ticker="KXSTATEDEEPDJT")
response

{'markets': [{'ticker': 'KXSTATEDEEPDJT-24-R',
   'event_ticker': 'KXSTATEDEEPDJT-24-R',
   'market_type': 'binary',
   'title': 'Will Trump win a deep blue state?',
   'subtitle': '',
   'yes_sub_title': 'Yes',
   'no_sub_title': 'Yes',
   'open_time': '2024-10-25T14:00:00Z',
   'close_time': '2025-11-05T15:00:00Z',
   'expected_expiration_time': '2025-01-07T15:00:00Z',
   'expiration_time': '2025-11-05T15:00:00Z',
   'latest_expiration_time': '2025-11-05T15:00:00Z',
   'settlement_timer_seconds': 300,
   'status': 'active',
   'response_price_units': 'usd_cent',
   'notional_value': 100,
   'tick_size': 1,
   'yes_bid': 21,
   'yes_ask': 24,
   'no_bid': 76,
   'no_ask': 79,
   'last_price': 24,
   'previous_yes_bid': 20,
   'previous_yes_ask': 23,
   'previous_price': 23,
   'volume': 1218859,
   'volume_24h': 490210,
   'liquidity': 204415702,
   'open_interest': 722794,
   'result': '',
   'can_close_early': True,
   'expiration_value': '',
   'category': 'Elections',
   'risk_lim

In [10]:
# TODO Use this to get the data for the other states
response = exchange_client.get_markets(series_ticker="PRESPARTYCA")
response

{'markets': [{'ticker': 'PRESPARTYCA-24-R',
   'event_ticker': 'PRESPARTYCA-24',
   'market_type': 'binary',
   'title': 'Will the Republican party win California in the presidential election?',
   'subtitle': ':: or another Republican',
   'yes_sub_title': 'Donald Trump',
   'no_sub_title': 'Donald Trump',
   'open_time': '2024-10-11T14:00:00Z',
   'close_time': '2025-11-05T15:00:00Z',
   'expected_expiration_time': '2025-01-07T15:00:00Z',
   'expiration_time': '2025-11-05T15:00:00Z',
   'latest_expiration_time': '2025-11-05T15:00:00Z',
   'settlement_timer_seconds': 14,
   'status': 'active',
   'response_price_units': 'usd_cent',
   'notional_value': 100,
   'tick_size': 1,
   'yes_bid': 3,
   'yes_ask': 4,
   'no_bid': 96,
   'no_ask': 97,
   'last_price': 4,
   'previous_yes_bid': 2,
   'previous_yes_ask': 4,
   'previous_price': 4,
   'volume': 888325,
   'volume_24h': 250975,
   'liquidity': 460962894,
   'open_interest': 622267,
   'result': '',
   'can_close_early': True,
   '

In [3]:
# Solver for the linear program
import numpy as np
from scipy.optimize import linprog

from src.np_utils import enumerate_binary_inputs

def make_event_matrix(n: int) -> np.ndarray:
    # last event is product of first n-1 events
    event_mat = np.empty((2 ** (n - 1), n), dtype=np.int8)
    event_mat[:, :-1] = enumerate_binary_inputs(n-1, dtype=event_mat.dtype)
    event_mat[:, -1] = np.prod(event_mat[:, :-1], axis=1)
    return event_mat

def make_bound_matrix(event_matrix: np.ndarray, cost_weights: np.ndarray) -> np.ndarray:
    n = event_matrix.shape[1]
    # Result should be 2^(n-1) x 2n
    bound_matrix = np.empty((event_matrix.shape[0], 2*n), dtype=cost_weights.dtype)
    bound_matrix[:, :n] = event_matrix - cost_weights[None, :n]
    bound_matrix[:, n:] = 1 - event_matrix - cost_weights[None, n:]
    return bound_matrix




In [ ]:
# Test things out first
n_events = 4
min_profit_rate=0.625
# cost_weights = np.full(n_events * 2, 0.3, dtype=np.float16)
cost_weights = np.array([0.1, 0.1, 0.1, 0.5, 0.9, 0.9, 0.9, 0.5], dtype=np.float16) # Test for n_events = 4
event_matrix = make_event_matrix(n_events)
bound_matrix = make_bound_matrix(event_matrix, cost_weights)

In [100]:
print(cost_weights)
print(event_matrix)
print(bound_matrix)

[0.1 0.1 0.1 0.5 0.9 0.9 0.9 0.5]
[[0 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 1 1 0]
 [1 0 0 0]
 [1 0 1 0]
 [1 1 0 0]
 [1 1 1 1]]
[[-0.1        -0.1        -0.1        -0.5         0.10000002  0.10000002
   0.10000002  0.5       ]
 [-0.1        -0.1         0.9        -0.5         0.10000002  0.10000002
  -0.9         0.5       ]
 [-0.1         0.9        -0.1        -0.5         0.10000002 -0.9
   0.10000002  0.5       ]
 [-0.1         0.9         0.9        -0.5         0.10000002 -0.9
  -0.9         0.5       ]
 [ 0.9        -0.1        -0.1        -0.5        -0.9         0.10000002
   0.10000002  0.5       ]
 [ 0.9        -0.1         0.9        -0.5        -0.9         0.10000002
  -0.9         0.5       ]
 [ 0.9         0.9        -0.1        -0.5        -0.9        -0.9
   0.10000002  0.5       ]
 [ 0.9         0.9         0.9         0.5        -0.9        -0.9
  -0.9        -0.5       ]]


In [101]:
res = linprog(
    cost_weights, 
    A_ub=-bound_matrix, # Negative because this computes negative profit
    b_ub=np.full(bound_matrix.shape[0], fill_value=-min_profit_rate), # Negative because we want to lower bound profit
    A_eq=cost_weights[None, :],
    b_eq=np.array([1], dtype=cost_weights.dtype),  # We are constrained to spending $1
    bounds=(0, None)
)

res.message

'Optimization terminated successfully. (HiGHS Status 7: Optimal)'

In [102]:
yes = res.x[:n_events]
no = res.x[n_events:]

print(yes)
print(no)
print(res.fun)

[0.06249999 0.         1.56250005 0.        ]
[ 0.06249999 -0.          0.          1.56250001]
0.9999999999999999


In [103]:
# Min profit achieved by solution
guaranteed_profit_rate = np.min(bound_matrix @ res.x) / (cost_weights @ res.x)
guaranteed_profit_rate

np.float64(0.6250000000000001)

In [44]:
# Now we test things out for real
DEEP_BLUE_STATES = ['CA', 'CO', 'CT', 'DE', 'HI', 'IL', 'MD', 'MA', 'NJ', 'NM', 'NY', 'OR', 'RI', 'VT', 'VA', 'WA']
DEEP_BLUE_STATE_TICKERS = [f"PRESPARTY{abbr}-24" for abbr in DEEP_BLUE_STATES]
DEEP_BLUE_TICKER = "KXSTATEDEEPDJT-24-R"

In [45]:
# Get the price for the deep blue bet
response = exchange_client.get_markets(tickers=DEEP_BLUE_TICKER)
# This is the price to buy "No, DJT does not win any deep blue states"
db_kam = response['markets'][0]['no_ask'] / 100
# This is the price to buy "Yes, DJT does win some deep blue states"
db_trump = response['markets'][0]['yes_ask'] / 100

pprint(response)
db_kam, db_trump

{'cursor': '',
 'markets': [{'can_close_early': True,
              'category': 'Elections',
              'close_time': '2025-11-05T15:00:00Z',
              'event_ticker': 'KXSTATEDEEPDJT-24-R',
              'expected_expiration_time': '2025-01-07T15:00:00Z',
              'expiration_time': '2025-11-05T15:00:00Z',
              'expiration_value': '',
              'last_price': 24,
              'latest_expiration_time': '2025-11-05T15:00:00Z',
              'liquidity': 203523640,
              'market_type': 'binary',
              'no_ask': 79,
              'no_bid': 76,
              'no_sub_title': 'Yes',
              'notional_value': 100,
              'open_interest': 728077,
              'open_time': '2024-10-25T14:00:00Z',
              'previous_price': 22,
              'previous_yes_ask': 22,
              'previous_yes_bid': 20,
              'response_price_units': 'usd_cent',
              'result': '',
              'risk_limit_cents': 0,
              'rules_

(0.79, 0.24)

In [46]:
# Get the price for the deep blue bet
kam_state_prices = []
trump_state_prices = []
for state, ticker in zip(DEEP_BLUE_STATES, DEEP_BLUE_STATE_TICKERS):
    response_d = exchange_client.get_markets(tickers=f"{ticker}-D")
    response_r = exchange_client.get_markets(tickers=f"{ticker}-R")
    # This is the price to buy "Yes, Kamala Harris wins this state"
    kam_price = min(response_d['markets'][0]['yes_ask'] / 100, response_r['markets'][0]['no_ask'] / 100)
    # This is the price to buy "Yes, DJT does win some deep blue states"
    trump_price = min(response_d['markets'][0]['no_ask'] / 100, response_r['markets'][0]['yes_ask'] / 100)

    # pprint(response)
    kam_state_prices.append(kam_price)
    trump_state_prices.append(trump_price)

for state, kam_price, trump_price in zip(DEEP_BLUE_STATES, kam_state_prices, trump_state_prices):
    print(f"{state}: Kamala Harris: {kam_price} Trump: {trump_price}")



CA: Kamala Harris: 0.97 Trump: 0.04
CO: Kamala Harris: 0.97 Trump: 0.05
CT: Kamala Harris: 0.97 Trump: 0.05
DE: Kamala Harris: 0.97 Trump: 0.04
HI: Kamala Harris: 0.96 Trump: 0.06
IL: Kamala Harris: 0.96 Trump: 0.06
MD: Kamala Harris: 0.98 Trump: 0.03
MA: Kamala Harris: 0.98 Trump: 0.04
NJ: Kamala Harris: 0.95 Trump: 0.07
NM: Kamala Harris: 0.9 Trump: 0.11
NY: Kamala Harris: 0.96 Trump: 0.05
OR: Kamala Harris: 0.96 Trump: 0.06
RI: Kamala Harris: 0.97 Trump: 0.05
VT: Kamala Harris: 0.98 Trump: 0.04
VA: Kamala Harris: 0.88 Trump: 0.13
WA: Kamala Harris: 0.97 Trump: 0.04


In [76]:
cost_weights = np.array(kam_state_prices + [db_kam] + trump_state_prices + [db_trump], dtype=np.float16)
# Alternatively we 
# cost_weights = np.array(kam_state_prices + [db_kam], dtype=np.float16)
# cost_weights = np.concatenate([cost_weights, 1- cost_weights])

assert len(cost_weights) % 2 == 0
n_events = len(cost_weights) // 2
print(f"Number of events: {n_events}")

# What if I edit the cost weights for deep blue
# cost_weights[n_events] = 0.5
# cost_weights[-1] = 0.5
print(f"Cost Weights: {cost_weights}")

min_profit_rate=0.00001
print(f"Min profit rate: {min_profit_rate}")

event_matrix = make_event_matrix(n_events)
bound_matrix = make_bound_matrix(event_matrix, cost_weights)
print(f"Event Matrix Shape: {event_matrix.shape}")
print(f"Event Matrix First Rows: {event_matrix[:10]}")
print(f"Event Matrix Dtype: {event_matrix.dtype}")
print(f"Bound Matrix Shape: {bound_matrix.shape}")
print(f"Bound Matrix First Rows: {bound_matrix[:10]}")
print(f"Bound Matrix Dtype: {bound_matrix.dtype}")

Number of events: 17
Cost Weights: [0.97 0.97 0.97 0.97 0.96 0.96 0.98 0.98 0.95 0.9  0.96 0.96 0.97 0.98
 0.88 0.97 0.79 0.04 0.05 0.05 0.04 0.06 0.06 0.03 0.04 0.07 0.11 0.05
 0.06 0.05 0.04 0.13 0.04 0.24]
Min profit rate: 1e-05
Event Matrix Shape: (65536, 17)
Event Matrix First Rows: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]]
Event Matrix Dtype: int8
Bound Matrix Shape: (65536, 34)
Bound Matrix First Rows: [[-0.97    -0.97    -0.97    -0.97    -0.96    -0.96    -0.98    -0.98
  -0.95    -0.9     -0.96    -0.96    -0.97    -0.98    -0.88    -0.97
  -0.79     0.96     0.95     0.95     0.96     0.94     0.94     0.97
   0.96     0.93     0.89     0.95     0.94  

In [77]:
res = linprog(
    cost_weights, 
    A_ub=-bound_matrix, # Negative because this computes negative profit
    b_ub=np.full(bound_matrix.shape[0], fill_value=-min_profit_rate), # Negative because we want to lower bound profit
    # A_eq=cost_weights[None, :],
    # b_eq=np.array([1], dtype=cost_weights.dtype),  # We are constrained to spending $1, this will force us to do something
    bounds=(0, None),
    method="highs"
)

res.message

'The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is Basic)'

In [78]:
yes = res.x[:n_events]
no = res.x[n_events:]

print(yes)
print(no)
print(res.fun)

TypeError: 'NoneType' object is not subscriptable

In [66]:
# Min profit achieved by solution
guaranteed_profit_rate = np.min(bound_matrix @ res.x.astype(np.float32)) / (cost_weights @ res.x)
guaranteed_profit_rate


np.float64(-0.009790024720132351)